In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

397


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_11월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,0,"['과천', '서울대', '공원', '딜쿠샤', '철도', '노조', '파업', '...","[('과천', 'Noun'), ('서울대', 'Noun'), ('공원', 'Noun..."
1,서울은 지하철 타기가 전쟁통을 방불케 한다,0,"['서울', '은', '지하철', '타기가', '전쟁', '통', '을', '방불'...","[('서울', 'Noun'), ('은', 'Josa'), ('지하철', 'Noun'..."
2,그래도 질서 있게 줄 섰다가 다음 열차를 타라는 안내 방송이 나오면 타기를 중단하고...,0,"['그래도', '질서', '있게', '줄', '섰다가', '다음', '열차', '를...","[('그래도', 'Adverb'), ('질서', 'Noun'), ('있다', 'Ad..."
3,서울대공원 안의 호수 메타세쿼이아 길 딜쿠샤는 기쁜 마음 이라는 뜻의 페르시아어로 ...,0,"['서울대', '공원', '안', '의', '호수', '메타세쿼이아', '길', '...","[('서울대', 'Noun'), ('공원', 'Noun'), ('안', 'Noun'..."
4,앨버트 테일러는 3 1운동을 뉴욕타임즈에 보도함으로써 우리 나라의 항일운동을 세계에...,0,"['앨버트', '테일러', '는', '3', '1', '운동', '을', '뉴욕타임...","[('앨버트', 'Noun'), ('테일러', 'Noun'), ('는', 'Josa..."
...,...,...,...,...
1084185,갑신정변이 실패하고 홍영식이 비참하게 죽은 뒤 민영익의 도움을 받은 알렌은 흉가가 ...,22958,"['갑신정변', '이', '실패하고', '홍영식', '이', '비참하게', '죽은'...","[('갑신정변', 'Noun'), ('이', 'Josa'), ('실패하다', 'Ad..."
1084186,제중원의 뜰에는 나무 한그루가 서 있었다,22958,"['제중원', '의', '뜰', '에는', '나무', '한그루', '가', '서',...","[('제중원', 'Noun'), ('의', 'Josa'), ('뜰', 'Noun')..."
1084187,박지원의 집앞에 있었던 나무 홍영식의 집앞에 있었던 나무 그날 길 잃은 아이들처럼 ...,22958,"['박지원', '의', '집앞', '에', '있었던', '나무', '홍영식', '의...","[('박지원', 'Noun'), ('의', 'Josa'), ('집앞', 'Noun'..."
1084188,그녀와 내가 헤어진 지금 이 모든 일이 과연 우연일 뿐이라고 생각하는가 21쪽 쉽게...,22958,"['그녀', '와', '내', '가', '헤어진', '지금', '이', '모든', ...","[('그녀', 'Noun'), ('와', 'Josa'), ('내', 'Noun'),..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,0,"['과천', '서울대', '공원', '딜쿠샤', '철도', '노조', '파업', '...","[('과천', 'Noun'), ('서울대', 'Noun'), ('공원', 'Noun...",과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
1,서울은 지하철 타기가 전쟁통을 방불케 한다,0,"['서울', '은', '지하철', '타기가', '전쟁', '통', '을', '방불'...","[('서울', 'Noun'), ('은', 'Josa'), ('지하철', 'Noun'...",서울 은 지하철 타기가 전쟁 통 을 방불 케 한다
2,그래도 질서 있게 줄 섰다가 다음 열차를 타라는 안내 방송이 나오면 타기를 중단하고...,0,"['그래도', '질서', '있게', '줄', '섰다가', '다음', '열차', '를...","[('그래도', 'Adverb'), ('질서', 'Noun'), ('있다', 'Ad...",그래도 질서 있게 줄 섰다가 다음 열차 를 타라 는 안내 방송 이 나오면 타기를 중...
3,서울대공원 안의 호수 메타세쿼이아 길 딜쿠샤는 기쁜 마음 이라는 뜻의 페르시아어로 ...,0,"['서울대', '공원', '안', '의', '호수', '메타세쿼이아', '길', '...","[('서울대', 'Noun'), ('공원', 'Noun'), ('안', 'Noun'...",서울대 공원 안 의 호수 메타세쿼이아 길 딜쿠샤 는 기쁜 마음 이라는 뜻 의 페르시...
4,앨버트 테일러는 3 1운동을 뉴욕타임즈에 보도함으로써 우리 나라의 항일운동을 세계에...,0,"['앨버트', '테일러', '는', '3', '1', '운동', '을', '뉴욕타임...","[('앨버트', 'Noun'), ('테일러', 'Noun'), ('는', 'Josa...",앨버트 테일러 는 3 1 운동 을 뉴욕타임즈 에 보도 함 으로써 우리 나라 의 항일...
...,...,...,...,...,...
1084185,갑신정변이 실패하고 홍영식이 비참하게 죽은 뒤 민영익의 도움을 받은 알렌은 흉가가 ...,22958,"['갑신정변', '이', '실패하고', '홍영식', '이', '비참하게', '죽은'...","[('갑신정변', 'Noun'), ('이', 'Josa'), ('실패하다', 'Ad...",갑신정변 이 실패하고 홍영식 이 비참하게 죽은 뒤 민영익 의 도움 을 받은 알렌 은...
1084186,제중원의 뜰에는 나무 한그루가 서 있었다,22958,"['제중원', '의', '뜰', '에는', '나무', '한그루', '가', '서',...","[('제중원', 'Noun'), ('의', 'Josa'), ('뜰', 'Noun')...",제중원 의 뜰 에는 나무 한그루 가 서 있었다
1084187,박지원의 집앞에 있었던 나무 홍영식의 집앞에 있었던 나무 그날 길 잃은 아이들처럼 ...,22958,"['박지원', '의', '집앞', '에', '있었던', '나무', '홍영식', '의...","[('박지원', 'Noun'), ('의', 'Josa'), ('집앞', 'Noun'...",박지원 의 집앞 에 있었던 나무 홍영식 의 집앞 에 있었던 나무 그날 길 잃은 아이...
1084188,그녀와 내가 헤어진 지금 이 모든 일이 과연 우연일 뿐이라고 생각하는가 21쪽 쉽게...,22958,"['그녀', '와', '내', '가', '헤어진', '지금', '이', '모든', ...","[('그녀', 'Noun'), ('와', 'Josa'), ('내', 'Noun'),...",그녀 와 내 가 헤어진 지금 이 모든 일이 과연 우연 일 뿐 이라고 생각 하는가 2...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,0,"['과천', '서울대', '공원', '딜쿠샤', '철도', '노조', '파업', '...","[('과천', 'Noun'), ('서울대', 'Noun'), ('공원', 'Noun...",과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
1,서울은 지하철 타기가 전쟁통을 방불케 한다,0,"['서울', '은', '지하철', '타기가', '전쟁', '통', '을', '방불'...","[('서울', 'Noun'), ('은', 'Josa'), ('지하철', 'Noun'...",서울 은 지하철 타기가 전쟁 통 을 방불 케 한다,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
2,그래도 질서 있게 줄 섰다가 다음 열차를 타라는 안내 방송이 나오면 타기를 중단하고...,0,"['그래도', '질서', '있게', '줄', '섰다가', '다음', '열차', '를...","[('그래도', 'Adverb'), ('질서', 'Noun'), ('있다', 'Ad...",그래도 질서 있게 줄 섰다가 다음 열차 를 타라 는 안내 방송 이 나오면 타기를 중...,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
3,서울대공원 안의 호수 메타세쿼이아 길 딜쿠샤는 기쁜 마음 이라는 뜻의 페르시아어로 ...,0,"['서울대', '공원', '안', '의', '호수', '메타세쿼이아', '길', '...","[('서울대', 'Noun'), ('공원', 'Noun'), ('안', 'Noun'...",서울대 공원 안 의 호수 메타세쿼이아 길 딜쿠샤 는 기쁜 마음 이라는 뜻 의 페르시...,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
4,앨버트 테일러는 3 1운동을 뉴욕타임즈에 보도함으로써 우리 나라의 항일운동을 세계에...,0,"['앨버트', '테일러', '는', '3', '1', '운동', '을', '뉴욕타임...","[('앨버트', 'Noun'), ('테일러', 'Noun'), ('는', 'Josa...",앨버트 테일러 는 3 1 운동 을 뉴욕타임즈 에 보도 함 으로써 우리 나라 의 항일...,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
...,...,...,...,...,...,...
1084185,갑신정변이 실패하고 홍영식이 비참하게 죽은 뒤 민영익의 도움을 받은 알렌은 흉가가 ...,22958,"['갑신정변', '이', '실패하고', '홍영식', '이', '비참하게', '죽은'...","[('갑신정변', 'Noun'), ('이', 'Josa'), ('실패하다', 'Ad...",갑신정변 이 실패하고 홍영식 이 비참하게 죽은 뒤 민영익 의 도움 을 받은 알렌 은...,하지만 네 즈 미 그건 정말 새였을까 내 가 본 게 확실할까 혹시 내 가 잘못 본 ...
1084186,제중원의 뜰에는 나무 한그루가 서 있었다,22958,"['제중원', '의', '뜰', '에는', '나무', '한그루', '가', '서',...","[('제중원', 'Noun'), ('의', 'Josa'), ('뜰', 'Noun')...",제중원 의 뜰 에는 나무 한그루 가 서 있었다,하지만 네 즈 미 그건 정말 새였을까 내 가 본 게 확실할까 혹시 내 가 잘못 본 ...
1084187,박지원의 집앞에 있었던 나무 홍영식의 집앞에 있었던 나무 그날 길 잃은 아이들처럼 ...,22958,"['박지원', '의', '집앞', '에', '있었던', '나무', '홍영식', '의...","[('박지원', 'Noun'), ('의', 'Josa'), ('집앞', 'Noun'...",박지원 의 집앞 에 있었던 나무 홍영식 의 집앞 에 있었던 나무 그날 길 잃은 아이...,하지만 네 즈 미 그건 정말 새였을까 내 가 본 게 확실할까 혹시 내 가 잘못 본 ...
1084188,그녀와 내가 헤어진 지금 이 모든 일이 과연 우연일 뿐이라고 생각하는가 21쪽 쉽게...,22958,"['그녀', '와', '내', '가', '헤어진', '지금', '이', '모든', ...","[('그녀', 'Noun'), ('와', 'Josa'), ('내', 'Noun'),...",그녀 와 내 가 헤어진 지금 이 모든 일이 과연 우연 일 뿐 이라고 생각 하는가 2...,하지만 네 즈 미 그건 정말 새였을까 내 가 본 게 확실할까 혹시 내 가 잘못 본 ...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,과천 서울대 공원 딜쿠샤 철도 노조 파업 2022 11 30 수요일 오늘 과천 사는...
1,1,서울 근교 의 강화도 에서 예쁜 카페 를 만났어요 멋진 아 뜰리 에가 있어서 그림 ...
2,2,안녕하세요 여러분 벌써 2022년 이 마무리 되어가는데요 얼마 전 에 상명대학교 계...
3,3,어느덧 오감 달팽이 가을 학기 마지막 수업 을 마쳤어요 그간 아팠던 쓰 롱 이 는 ...
4,4,시내 에 있어 접근성 이 뛰어난 치악산 국 형사 는 시민 들 이 자주 찾는 사찰 입...
...,...,...
22953,22954,안녕하셔요 제 가 또 돌아왔네요 10월 도 긴 것 같은데 기분 탓 인가 10월 첫날...
22954,22955,경주 불국사 템플스테이 후기 참된 나를 찾아가는 행복여행 원래 8월 말 에 휴가 시...
22955,22956,조계산 888 m 1 산림청 100 대 명산 2 한국 의 산하 100 대 명산 89...
22956,22957,현대 자동차 그룹 글로벌 비즈니스 센터 GBC 조감도 1 출처 글로벌 비즈니스 센터...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(22958, 492176)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03056396, 0.08417275, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02672624, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03878887, 0.02373868, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.024132  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,492166,492167,492168,492169,492170,492171,492172,492173,492174,492175
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.030564,0.084173,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.026726,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.066362,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22953,0.007114,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22954,0.014166,0.039012,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22955,0.038789,0.023739,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22956,0.000000,0.000000,0.024132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,0001014179,0001063211,0001063212,0001063213,0001063214,...,힛트비그,힛틀러,힝,힝구,힝로,힝캬쿠,힝클리,힝힝,힠,힣
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.030564,0.084173,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.026726,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.066362,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22953,0.007114,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22954,0.014166,0.039012,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22955,0.038789,0.023739,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22956,0.000000,0.000000,0.024132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/11월/나무문화_11월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [15]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

         ㄱ   ㄱㄱ  ㄱㄱㄱ  ㄱㄱㄹ   ㄱㄴ  ㄱㄴㄷ  ㄱㄴㄷㄹ  ㄱㄷㅎ  ㄱㄹㄹㅋㅈ   ㄱㅁ  ...  힛트비그  힛틀러  \
0      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
1      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
2      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
...    ...  ...  ...  ...  ...  ...   ...  ...    ...  ...  ...   ...  ...   
22953  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
22954  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
22955  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
22956  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   
22957  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0    0.0  0.0  ...   0.0  0.0   

         힝   힝구   힝로  힝캬쿠  힝클리   힝힝    힠    힣  
0      0.0  0.0

In [16]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,122.464332
1,00,312.586744
2,000,40.760484
3,0001,0.785293
4,00010,0.055470
...,...,...
492171,힝캬쿠,0.048638
492172,힝클리,0.048046
492173,힝힝,0.241280
492174,힠,0.008774


In [17]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/11월/나무문화_11월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')